This file generates a faux isotropic background that contains the same number of photons in both the true isotropic and anisotropic backgrounds

In [52]:
%reset

In [53]:
import sys
sys.path.append('/home/users/ids29/DGRB')

In [54]:
import aegis
import numpy as np
import torch
import healpy as hp
import pickle as pk
from astropy import units
from astropy import constants as c
import matplotlib.pyplot as plt
from os import listdir
import os
from sbi.inference import SNLE, SNPE#, prepare_for_sbi, simulate_for_sbi
from sbi import utils as utils
from sbi import analysis as analysis
# from sbi.inference.base import infer
from getdist import plots, MCSamples
from joblib import Parallel, delayed, parallel_backend
from scipy.integrate import quad, simpson
import pickle
from scipy.stats import norm
import sources.DMsignal as DMsignal
import sources.FermiBackgrounds as FermiBackgrounds
from astropy import units as u


%matplotlib inline

In [55]:
grains=10_000
num_workers = 48

In [56]:
energy_range = [1_000, 100_000] #MeV 
max_radius = 220 #kpc #This is the virial radius of the Milky Way
exposure = 2_000*10*0.2 #cm^2 yr
flux_cut = 1e-9 #photons/cm^2/s
angular_cut_degrees = 180 #degrees # 180 degress is full sky
lat_cut_degrees = 0 #degrees # 0 degrees is full sky
luminosity_range = np.array([1e13, 1e53]) # Minimum value set by considering source at distnace of closest approach by line of sight at 40 degrees and receiving 0.1 photon at detector side. CHANGE THIS FOR FINAL PROBLEM.
                                            # Maximum value set by that value of luminosity that suffieciently suppreses the luminoisity function at the higher end of the luminosity range.


parameter_range_aegis = [[], []]
abundance_luminosity_and_spectrum_list = []
source_class_list = []
parameter_names = []
energy_range_gen = [energy_range[0]*0.5, energy_range[1]*1.5]
angular_cut = angular_cut_degrees*u.deg.to('rad') # radians
angular_cut_gen =  min(angular_cut*1.5, np.pi) # radians
lat_cut = lat_cut_degrees*u.deg.to('rad') # radians
lat_cut_gen = lat_cut*0.5 # radians

In [57]:
def build_aegis_object():
    engine = aegis.aegis(abundance_luminosity_and_spectrum_list, source_class_list, parameter_range_aegis, energy_range, luminosity_range, max_radius, exposure, angular_cut , lat_cut, flux_cut, energy_range_gen=energy_range_gen, verbose = False)
    engine.angular_cut_gen, engine.lat_cut_gen = angular_cut_gen, lat_cut_gen
    return engine


In [58]:
# 1) Point to your data directory and choose a channel
data_dir = "/home/users/ids29/DGRB/data/dm_spectra/"         # must contain AtProduction_gammas.dat
channel  = "Tau"                           # e.g. 'b' (bottom quarks) or 'Tau'


# 2) Build the DMsignal object
dm = DMsignal.DMsignal(directory=data_dir, channel=channel)

Background signal

In [ ]:
# Note: Terminology:
# flux is photon/cm^2/s
# differential flux is photon/cm^2/s/MeV

# Add Fermi isotropic background source class
data_root = '/home/users/ids29/DGRB'
fb = FermiBackgrounds.FermiBackgrounds(data_root)

def spec_BG(energy, params): # returns spectrum dF/dE for each value of energy specified in the input array 'energy' 
    #such that the area under curve equals the flux (photons/cm^2/s) over the detector's energy range
    A_BG = params[0]
    dFdEdSr_E_interpfunc = fb.get_isotropic_background_spectrum_func() # returns an interpolation function -> dF/dE/d(solid_angle) versus E
    
    num_photons_iso_bg = ... # GENERATE THE FULL ISOTROPIC BACKGROUND USING AEGIS, AND COUNT THE TOTAL NUMBER OF PHOTONS # for full sky in detector's energy range
    num_photons_aniso_bg = ... # GENERATE THE FULL ANISOTROPIC BACKGROUND USING AEGIS, AND COUNT THE TOTAL NUMBER OF PHOTONS # for full sky in detector's energy range
    correction_normalization = (num_photons_iso_bg + num_photons_aniso_bg) / num_photons_iso_bg # this factor adjusts the normalization such that the number of photons in this
                                # faux isotropic background is equal to the sum of number of photons in the Fermi isotropic background, and the number of photons in the
                                # Fermi non-isotropic background, everthing measured in the region of interest defined by the angular cut and latitudinal cut.
                                
    return A_BG * 4*np.pi * dFdEdSr_E_interpfunc(energy) * correction_normalization # 4*pi converts the term dF/dE/d(solid angle) to dF/dE since dF/dE/d(solid angle) is constant
                                                                                    # in every direction

# AEGIS functions for Fermi isotropic background
als_FIB = [spec_BG]

In [60]:
# a simple simulator with the total number of photons as the summary statistic
def simulator(my_AEGIS, params):

    input_params = params.numpy()

    source_info = my_AEGIS.create_sources(input_params, grains=grains, epsilon=1e-3)
    photon_info = my_AEGIS.generate_photons_from_sources(input_params, source_info, grains=grains) 
    obs_info = {'psf_fits_path': '/home/users/ids29/DGRB/FERMI_files/psf_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'edisp_fits_path': '/home/users/ids29/DGRB/FERMI_files/edisp_P8R3_ULTRACLEANVETO_V2_PSF.fits', 'event_type': 'PSF3', 'exposure_map': None}
    obs_photon_info = my_AEGIS.mock_observe(photon_info, obs_info)
    
    return obs_photon_info

Load Test parameters

In [61]:
# ────────────────────────────────────────────────────────────────────────
# 3)  Ask the object for its default model parameters
#     (mass, amplitude, background).  These numbers come straight from
#     the get_default_model() method you showed.
# ────────────────────────────────────────────────────────────────────────
model_defaults = dm.get_default_model()        # {'A_DM': …, 'A_BG': …, 'mass_DM_MeV': …}
mass_DM_MeV_test    = model_defaults["mass_DM_MeV"]  # MeV
mass_DM_GeV_test    = mass_DM_MeV_test / 1e3                  # GeV
A_DM_test           = model_defaults["A_DM"] # unitless
A_BG_test           = model_defaults["A_BG"]

print(f"A_DM_test = {A_DM_test}; mass_DM_GeV_test = {mass_DM_GeV_test}; A_BG_test = {A_BG_test}")

A_DM_test = 200.0; mass_DM_GeV_test = 200.0; A_BG_test = 1.0


Generate Test Data

In [62]:
# params_DM_test = torch.tensor([A_DM_test, mass_DM_GeV_test])
params_BG_test = torch.tensor([A_BG_test])

# Instatantiate the AEGIS object for DM and Background
obj_AEGIS = build_aegis_object()

# # Configure the AEGIS object for DM
# obj_AEGIS.abun_lum_spec = [als_DM]
# obj_AEGIS.source_class_list = ['isotropic_faint_single_spectrum']

# # Run the simulator for DM data
# photon_info_DM_test = simulator(obj_AEGIS, params_DM_test)

# Configure the AEGIS object for Background
obj_AEGIS.abun_lum_spec =  [als_FIB]
obj_AEGIS.source_class_list = ['isotropic_diffuse']

# Run the simulator for Background data
photon_info_BG_test = simulator(obj_AEGIS, params_BG_test)

# Save the photon information and theta parameters to files.
# with open(f'test_data_DM.pkl', 'wb') as f:
#     pickle.dump(photon_info_DM_test, f)
with open(f'test_data_BG.pkl', 'wb') as f:
    pickle.dump(photon_info_BG_test, f)

# torch.save(params_DM_test, f'test_thetas_DM.pt')
torch.save(params_BG_test, f'test_thetas_BG.pt')